# Downloading, converting, and subsetting Forest Service data
* Author: Katie Murenbeeld
* Date: 19 Nov 2019
* Updated: 09 Dec 2019

## Introduction:
Within this notebook I will show you:
1. Where to retrieve and download Forest Service activity data 
2. How to convert the data to a .csv file
3. How to "subset" the data for the desired state and information
4. How to convert the planned and completed dates to serial dates
5. How to reclassify the treatment activities.
6. How to calculate Time Lag (the difference between completed and planned date) and the difference in planned and completed area treated and store in a new column
7. Write out a .csv file with the modified dataset


### 1. Retrieve and download Forest Service activity data
Forest Service activity data can be found at here: https://data.fs.usda.gov/geodata/edw/datasets.php

Please chose the dataset you would like to work with (in this notebook I will be using the Timber Harvest data) by clicking on the shapefile link for the desired dataset (insert screen shot here). Clicking on the link will automatically start the download. Once download is complete, move the dataset to your desired location. 

* Note: The metadata for the file is also useful and will define the different code names and numbers.

After you move the downloaded data, set the directories for your raw data and for the resulting .csv files.

In [1]:
# Set directories

raw_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/RAW/'
mod_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/MOD/'

### 2. Convert the data to a .csv file
Remember, the file you downloaded is a shape file. There are several files that comprise a GIS/ESRI shapefile. One of these is a .dbf. The .dbf is the file that will be converted to a .csv file. The conversion will produce a rather large .csv that should be stored in your modified data directory (mod_data_dir). To convert the data you will need a specific library called _simpledbf_ and the tool _Dbf5_ from _simpledbf_.

Before starting, I recommend that you rename the .dbf to reflect when you downloaded the raw data.

In [2]:
# Download the relevant libraries
import simpledbf 
from simpledbf import Dbf5 

# Other important libraries
import pandas as pd #For working with dataframes
import numpy as np #For working with arrays
import datetime as dt #For converting dates to serial datetimes


In [3]:
# Convert the most up to date timber harvest data from dbf to csv
# It is useful to use a name that reflects the general time that you
# downloaded the data.

#dbf = Dbf5(raw_data_dir + 'S_USA_Activity_TimberHarvest_july2019.dbf', codec='utf-8') # Import the dbf and assign the code 

#dbf.to_csv(mod_data_dir + 'USA_TH_JULY2019.csv') # Covert to csv

In [4]:
# Use pandas to create dataframe from new csv
df = pd.read_csv(mod_data_dir + 'USA_TH_JULY2019.csv').reset_index()
df.head(1) # Show the first row of the dataframe as a check.

,index,ADMIN_FORE,ADMIN_REGI,ADMIN_FO_1,PROCLAIMED,ADMIN_DIST,ADMIN_DI_1,HOME_ORG,ACTIVITY_U,SUID,...,DATA_SOU_1,ACCURACY,CRC_VALUE,UK,EDW_INSERT,ETL_MODIFI,REV_DATE,GIS_ACRES,SHAPE_AREA,SHAPE_LEN
0,0,6,5,Lassen National Forest,506,Almanor Ranger District,51,50651,50651,050651T376000076000,...,NaN,0.0,NaN,NaN,2019-07-01,2019-07-01,2018-01-09,25.759,0.000011,0.026648


### 3. Subset the data for the desired area (state, region, or forest) of interest.
As you can see, there are many columns with this data. Review the metadata to decide which columns will be most useful. In this case, I only want to look at columns:
1. STATE_ABBR, the state 
2. ADMIN_FORE, the number for associated with a National Forest (sometimes the name is missing from the dataset)
3. ADMIN_FO_1, the name of the National Forest
4. NEPA_DOC_N, the name of a Forest Service project, which are comprised of different activities.
5. ACTIVITY_N, the name of the activity
6. ACTIVITY_2, may be ACTIVITY_C in other datasets, the numerical activity code
7. NBR_UNITS_, the number of acres planned to be treated by an activity
8. NBR_UNITS1, the number of acres actually treated by an activity
9. DATE_PLANN, the date an activity was planned to be completed
10. DATE_COMPL, the date an activity was actually completed

One could also include ADMIN_REGI to look at Forest Service administrative regions.

For this example, I will then subset the reduced dataframe by state (i.e. Idaho). 

In [5]:
# Reduce the columns from the dataframe
df2 = df[['STATE_ABBR','ADMIN_FORE','ADMIN_FO_1','NEPA_DOC_N','ACTIVITY_N',
         'ACTIVITY_2','NBR_UNITS_','NBR_UNITS1','DATE_PLANN','DATE_COMPL']].copy()

# .copy() is important...

df2.head(1) # Check that the correct columns are in the new dataframe

,STATE_ABBR,ADMIN_FORE,ADMIN_FO_1,NEPA_DOC_N,ACTIVITY_N,ACTIVITY_2,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
0,CA,6,Lassen National Forest,(PALS)CREEKS II FOREST RESTORATION PROJECT FEI...,Commercial Thin,4220,26.0,26.0,2018-03-01,NaN


Finally, create a new data frame with only the data for Idaho. Use the .loc function in pandas to essentially slice the data by state.

In [6]:
# Use .loc and slice where column STATE_ABBR is equal to ID.
df_id = df2.loc[df2['STATE_ABBR']=='ID'].copy()
df_id.head(5) # Check the new dataframe

,STATE_ABBR,ADMIN_FORE,ADMIN_FO_1,NEPA_DOC_N,ACTIVITY_N,ACTIVITY_2,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
4,ID,17,Nez Perce - Clearwater National Forest,DEFAULT FOR NOT REQUIRED,Shelterwood Establishment Cut (with or without...,4131,6.0,6.0,1987-07-01,1987-07-01
14,ID,4,Idaho Panhandle National Forests,DEFAULT FOR NOT REQUIRED,"Salvage Cut (intermediate treatment, not regen...",4231,32.0,32.0,1968-06-01,1968-06-01
36,ID,4,Idaho Panhandle National Forests,DEFAULT FOR NOT REQUIRED,Stand Clearcut (EA/RH/FH),4113,6.0,6.0,1985-07-01,1985-07-01
49,ID,4,Idaho Panhandle National Forests,DEFAULT FOR NOT REQUIRED,Stand Clearcut (EA/RH/FH),4113,18.0,18.0,1969-06-01,1969-06-01
50,ID,4,Idaho Panhandle National Forests,DEFAULT FOR NOT REQUIRED,"Salvage Cut (intermediate treatment, not regen...",4231,29.0,29.0,1953-08-01,1953-08-01


### 4. Convert the planned and completed dates to serial dates.
In order to analyze this data through time, the current date data must be converted into serial dates using the datetime library. A new column will be added to the data frame for the new serial dates.

In [7]:
# Convert DATE_* columns to datetime. Add a new column for the serial dates

# Create a numpy array of all the Date Planned values
id_datearrP = df_id['DATE_PLANN'].values 
# Convert the values in that array to a serial datetime and name as a new variable
id_datearr_dtP = pd.to_datetime(id_datearrP) 
# Add a new column to the dataframe of the converted datetime
df_id['SerDatesPlan'] = id_datearr_dtP 

# Repeat for the Date Completed

# Create a numpy array of all the Date Completed values
id_datearrC = df_id['DATE_COMPL'].values
id_datearr_dtC = pd.to_datetime(id_datearrC)
df_id['SerDatesComp'] = id_datearr_dtC

df_id.head(1) # Check to make sure the new columns show up

,STATE_ABBR,ADMIN_FORE,ADMIN_FO_1,NEPA_DOC_N,ACTIVITY_N,ACTIVITY_2,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp
4,ID,17,Nez Perce - Clearwater National Forest,DEFAULT FOR NOT REQUIRED,Shelterwood Establishment Cut (with or without...,4131,6.0,6.0,1987-07-01,1987-07-01,1987-07-01,1987-07-01


Other useful columns to add include:
1. A column for year planned ('YP')
2. A column for year completed ('YC')

This will be useful for looking at the total number of times a treatment occurred through time and the total area completed through time. By looking at yearly totals, the resulting figures will look better.

In [8]:
# Use datetime to retrieve the year values.

df_id['YP'] = df_id['SerDatesPlan'].dt.year
df_id['YC'] = df_id['SerDatesComp'].dt.year

# Check that the two new columns showed up and are correct.
df_id.head(1)

# Note, the results may need to be an integer not a float and vice versa. 
# Some activities have not been completed, so YC will have some Nan values
# which makes the entire YC column data floats. Due to the Nans you may recieve an error
# in which case Nans may need to go to 0 and then be converted to float or integer.

# If need be, convert floats to integers.
# Check the data type using pandas .dtype
df_id['YC'].dtype

dtype('float64')

In [9]:
# Replace Nan with 0 if desired.


# Convert float to integer
#df_id.astype({'YC': 'int64'}).dtypes 

# Check data type again
#df_id['YC'].dtype

### 5. Reclassify the treatment activities.

Reclassify the treatment activites as you see fit using the metadat from the Forest Service data website to inform you (https://data.fs.usda.gov/geodata/edw/datasets.php). In this case the activities were reclassified into 14 groups based on the objective of the activity and the fraction of trees that are retained.

In [12]:
# Group some of the activities together for later use. Use np.select(conditions, choices, default=). 
# Set your conditions to the ACTIVITY_2 codes, use isin() for when multiple codes will be grouped.

conditions = [
    (df_id['ACTIVITY_2']==4113), 
    (df_id['ACTIVITY_2']==4177),
    (df_id['ACTIVITY_2']==4111),
    (df_id['ACTIVITY_2']==4175),
    (df_id['ACTIVITY_2']==4152),
    (df_id['ACTIVITY_2']==4151),
    (df_id['ACTIVITY_2']==4220),
    (df_id['ACTIVITY_2'].isin([4231,4232,4210,4211])),
    (df_id['ACTIVITY_2'].isin([4102,4121,4131,4132])),
    (df_id['ACTIVITY_2'].isin([4192,4194,4162])),
    (df_id['ACTIVITY_2'].isin([4141,4142,4145,4146])),
    (df_id['ACTIVITY_2'].isin([4148,4196,4183,4193])),
    (df_id['ACTIVITY_2']==4143),
    (df_id['ACTIVITY_2']==4242)
]
choices = ['SCC','A2_SCC','PCC','A2_PCC','GT_CUT','ST_CUT','COMM_THIN','INT_CUT','REGEN_PREP','REGEN_PREP10','REGEN_REMOVE','REGEN_REMOVE10','OVERSTORE_CUT','HARV_NOSTOCK']

In [13]:
# Use np.select() to create a new column with the coded/grouped activities.

df_id['NEW_CODE'] = np.select(conditions, choices, default='none')
df_id.head(2) # check the data

,STATE_ABBR,ADMIN_FORE,ADMIN_FO_1,NEPA_DOC_N,ACTIVITY_N,ACTIVITY_2,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC,NEW_CODE
4,ID,17,Nez Perce - Clearwater National Forest,DEFAULT FOR NOT REQUIRED,Shelterwood Establishment Cut (with or without...,4131,6.0,6.0,1987-07-01,1987-07-01,1987-07-01,1987-07-01,1987,1987.0,REGEN_PREP
14,ID,4,Idaho Panhandle National Forests,DEFAULT FOR NOT REQUIRED,"Salvage Cut (intermediate treatment, not regen...",4231,32.0,32.0,1968-06-01,1968-06-01,1968-06-01,1968-06-01,1968,1968.0,INT_CUT


### 6. Calculate the _time lag_ from planned to completed and the difference in area treated from planned to completed.
Use simple subtraction to calculated _time lag_ and the difference in area. Add these new values as new columns to the dataframe.

In [14]:
df_id['TIME_LAG'] = (df_id['SerDatesComp'] - df_id['SerDatesPlan']).dt.days
df_id.head(1)

,STATE_ABBR,ADMIN_FORE,ADMIN_FO_1,NEPA_DOC_N,ACTIVITY_N,ACTIVITY_2,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC,NEW_CODE,TIME_LAG
4,ID,17,Nez Perce - Clearwater National Forest,DEFAULT FOR NOT REQUIRED,Shelterwood Establishment Cut (with or without...,4131,6.0,6.0,1987-07-01,1987-07-01,1987-07-01,1987-07-01,1987,1987.0,REGEN_PREP,0.0


In [15]:
df_id['AREA_DIFF'] = (df_id['NBR_UNITS1'] - df_id['NBR_UNITS_'])
df_id.head(2)

,STATE_ABBR,ADMIN_FORE,ADMIN_FO_1,NEPA_DOC_N,ACTIVITY_N,ACTIVITY_2,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC,NEW_CODE,TIME_LAG,AREA_DIFF
4,ID,17,Nez Perce - Clearwater National Forest,DEFAULT FOR NOT REQUIRED,Shelterwood Establishment Cut (with or without...,4131,6.0,6.0,1987-07-01,1987-07-01,1987-07-01,1987-07-01,1987,1987.0,REGEN_PREP,0.0,0.0
14,ID,4,Idaho Panhandle National Forests,DEFAULT FOR NOT REQUIRED,"Salvage Cut (intermediate treatment, not regen...",4231,32.0,32.0,1968-06-01,1968-06-01,1968-06-01,1968-06-01,1968,1968.0,INT_CUT,0.0,0.0


In [16]:
# Optional: drop the DATE_PLANN and DATE_COMPL columns to clean table up more.
df_id = df_id.drop(['DATE_PLANN', 'DATE_COMPL'], axis=1)
df_id.head(2) # Check that the columns were dropped

,STATE_ABBR,ADMIN_FORE,ADMIN_FO_1,NEPA_DOC_N,ACTIVITY_N,ACTIVITY_2,NBR_UNITS_,NBR_UNITS1,SerDatesPlan,SerDatesComp,YP,YC,NEW_CODE,TIME_LAG,AREA_DIFF
4,ID,17,Nez Perce - Clearwater National Forest,DEFAULT FOR NOT REQUIRED,Shelterwood Establishment Cut (with or without...,4131,6.0,6.0,1987-07-01,1987-07-01,1987,1987.0,REGEN_PREP,0.0,0.0
14,ID,4,Idaho Panhandle National Forests,DEFAULT FOR NOT REQUIRED,"Salvage Cut (intermediate treatment, not regen...",4231,32.0,32.0,1968-06-01,1968-06-01,1968,1968.0,INT_CUT,0.0,0.0


### 7. Write to .csv file
Write the final version of the data frame to a .csv file and store in the mod_data_dir. Name in such a way that you will know this is for timber harvests in Idaho.

In [17]:
# Use pandas .to_csv to convert a dataframe to a .csv file. 
# The default encoding is utf-8, but this can be adjusted if desired.

df_id.to_csv(mod_data_dir + 'ID_TH_20191209_shiny.csv')